In [1]:
import numpy as np
import copy
from queue import PriorityQueue

In [2]:
# This function reads input file and place initial state and goal state into array
# Note the we use 2D numpy array in this project to store the 8-puzzle problem states
# We return two arrays start and goal representing start state and goal state

def read_input(file_name):
    f = open(file_name,'r')

    start = np.zeros((3, 3))
    goal = np.zeros((3, 3))

    lines = f.readlines()

    for i in range(len(lines)):
        if i in [0,1,2]:
            line = lines[i]
            line_list = line.split()
            for j in range(3):
                start[i][j] = int(line_list[j])
        if i in [4,5,6]:
            line = lines[i]
            line_list = line.split()
            for j in range(3):
                goal[i-4][j] = int(line_list[j])
    f.close()
    return start, goal

In [3]:
# This function compute the Manhattan distance between two states
def Manhattan(A, B):
    distance = 0
    for i in range(3):
        for j in range(3):
            if A[i][j] == B[i][j]:
                continue
            elif A[i][j] != 0:
                for ii in range(3):
                    for jj in range(3):
                        if A[i][j] == B[ii][jj]:
                            distance += abs(i-ii)+abs(j-jj)
    return distance


# The following two functions together calculate S(n) in the Nilsson's sequence score
def around(curr):
    long = ""
    for i in curr:
        for j in i:
            long+=(str(int(j)))
    around = long[0:3]+long[5]+long[8]+long[7]+long[6]+long[3]+long[0]
    return around

def sn(curr, goal):
    score = 0
    if curr[1][1] != goal[1][1]:
        score += 1
    a = around(curr)
    b = around(goal)
    for i in range(8):
        aa = a[i]+a[i+1]
        if aa[0] in b and aa not in b:
            score += 2
    return score


# For convenience, we not give two functions h1, h2 to calculate heuristic function values

def h1(A, B):
    return Manhattan(A, B)

def h2(A, B):
    return Manhattan(A, B) + 3*sn(A, B)


In [4]:
# Now that we know that each state is a 2d numpy array, we use TreeNode to represent each node
# where self.data is the 2d array

class TreeNode:
    def __init__(self):
        self.data = None
        self.f_value = None
        self.depth = None
        self.movement = None  # the movement of the blank tile to get the current state from its parent node state
        self.parent = None
        
    def equal(self, other):
        return np.array_equal(self.data, other.data)
    
    # Since TreeNode objects cannot be compared directly, we define a less than comparison criterion
    # If f_value can be compared, the problem is solved
    # If two TreeNode objects share the same f_value, then we should pop a random one
    # Here, we manually set the self one to be the larger one, this makes no difference
    def __lt__(self, other):
        if self.f_value != other.f_value:
            return self.f_value < other.f_value
        else:
            return self.data[0][0]+100 < other.data[0][0]

In [5]:
# This function considers all the possible movements of the given node
# The function returns all the possible children of the given node 
# along with the corresponding movements of the blank tile
# Note that at this stage we do not deal with the repeated states

def expand_node(node):
    children = []
    X = node.data
    
    # Consider whether the blank tile can move down
    for (i,j) in [(0,0),(0,1),(0,2),(1,0),(1,1),(1,2)]:
        if X[i][j] == 0:
            child1 = TreeNode()
            data1 = copy.deepcopy(X)
            data1[i][j], data1[i+1][j] = data1[i+1][j], data1[i][j]
            child1.data = data1
            child1.movement = 'D'
            children.append(child1)
    
    # Consider whether the blank tile can move up
    for (i,j) in [(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)]:
        if X[i][j] == 0:
            child2 = TreeNode()
            data2 = copy.deepcopy(X)
            data2[i][j], data2[i-1][j] = data2[i-1][j], data2[i][j]
            child2.data = data2
            child2.movement = 'U'
            children.append(child2)
            
    # Consider whether the blank tile can move right
    for (i,j) in [(0,0),(0,1),(1,0),(1,1),(2,0),(2,1)]:
        if X[i][j] == 0:
            child3 = TreeNode()
            data3 = copy.deepcopy(X)
            data3[i][j], data3[i][j+1] = data3[i][j+1], data3[i][j]
            child3.data = data3
            child3.movement = 'R'
            children.append(child3)

    # Consider whether the blank tile can move left
    for (i,j) in [(0,1),(0,2),(1,1),(1,2),(2,1),(2,2)]:
        if X[i][j] == 0:
            child4 = TreeNode()
            data4 = copy.deepcopy(X)
            data4[i][j], data4[i][j-1] = data4[i][j-1], data4[i][j]
            child4.data = data4
            child4.movement = 'L'
            children.append(child4)
    
    for child in children:
        child.depth = node.depth + 1
        child.parent = node
        
    return children
   

In [6]:
# Now we implement the A* Search Algorithm to solve the 8-puzzle problem

def AstarSearch(start_node, goal_node, heuristic):
    
    # We use a priority queue to store the nodes, where the f value indicates the priority order
    q = PriorityQueue()
    q.put((start_node.f_value, start_node))
    
    # We use a list named state_space to record the state space, so that we can aviod generating repeated states
    # In addition, by examining the size of the state_space list, we get the total number of the nodes generated
    state_space = [start_node.data]
    
    f_value_list = [start_node.f_value]
    movements = []
    
    (f, cur_node) = q.get()
    
    
    while not cur_node.equal(goal_node):
        
        children = expand_node(cur_node)
        
        for child in children:
            flag = True
            for state in state_space:
                if np.array_equal(state, child.data):
                    flag = False
            if flag == True:
                state_space.append(child.data)
                
                # Below line calculate the f(n) 
                # with g(n) = depth (which represents the path cost) 
                # and h(n) calculated by the heuristic function passed
                child.f_value = child.depth + heuristic(child.data, goal_node.data)
                
                q.put((child.f_value, child))
        
        (f, cur_node) = q.get()
     
    goal_depth = cur_node.depth
    
    N = len(state_space)
    
    actions = [cur_node.movement]
    x = cur_node
    while x.parent is not None:
        x = x.parent
        if x.movement is not None:
            actions.append(x.movement)
    actions.reverse()
    
    f_value_list = [cur_node.f_value]
    y = cur_node
    while y.parent is not None:
        y = y.parent
        f_value_list.append(y.f_value)
    f_value_list.reverse()
    
    
    return goal_depth, N, actions, f_value_list

In [7]:
# This function writes output file
def write_output(file_name, start, goal, goal_depth, N, actions, f_value_list):
    
    f = open(file_name, 'w')
    
    for i in range(3):
        line = ''
        for j in range(3):
            line += str(int(start[i][j]))
            if j == 2:
                line += '\n'
                f.write(line)
            else:
                line += ' '
    f.write('\n')
    
    for i in range(3):
        line = ''
        for j in range(3):
            line += str(int(goal[i][j]))
            if j == 2:
                line += '\n'
                f.write(line)
            else:
                line += ' '
    f.write('\n')
    
    f.write(str(goal_depth) +'\n')
    
    f.write(str(N) +'\n')
    
    actions_line = ' '.join(actions)
    f.write(actions_line + '\n')
    
    f_value_list_line = ' '.join(str(e) for e in f_value_list)
    f.write(f_value_list_line + '\n')
        
    f.close()

In [9]:
# Now that we've done with all the implememt, we now use the algorithms above to execute on the three input files

input_files = ['input1.txt','input2.txt','input3.txt']

heuristic_functions = [h1, h2]


for input_file in input_files:
    
    start_data, goal_data = read_input(input_file)

    start_node = TreeNode()
    start_node.depth = 0
    start_node.data = start_data

    goal_node = TreeNode()
    goal_node.data = goal_data
    
    for h in heuristic_functions:

        start_node.f_value = h(start_node.data, goal_node.data)

        goal_depth, N, actions, f_value_list = AstarSearch(start_node, goal_node, h)
        
        output_file = 'output' + input_file[5]
        if h == h1:
            output_file += 'h1.txt'
        else:
            output_file += 'h2.txt'
        
        write_output(output_file, start_data, goal_data, goal_depth, N, actions, f_value_list)